In [ ]:
# Note:
# The database containing the vectors for the OHS Act chunks has already been created on Weaviate.
# The vector database is called: OHS_ACT_VDB

In [1]:
#!pip install -U weaviate-client
!pip install "weaviate-client==3.*" # Version 4 of the client does not work

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 13.5 MB/s eta 0:00:00


In [32]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.0 MB/s eta 0:00:00


In [35]:
import pandas as pd
import numpy as np
import os

import weaviate

In [36]:
WCS_ENDPOINT = "https://my-sandbox1-1486fdzz.weaviate.network/"
WCS_API_KEY = "YOUR-API-KEY"
OPENAI_API_KEY = 'YOUR-API-KEY'

VECTOR_DB_NAME = 'OHS_ACT_VDB'

In [37]:
base_path = '../input/movie-summaries-cmu/'

In [38]:
os.listdir('../input/movie-summaries-cmu/')

['singapore.txt',
 'sample-data.csv',
 'ohs-act-data',
 'simplified_cmu_movie_data.csv',
 'singapore_rev1.txt',
 'MovieSummaries',
 'gospel-csv-data']

## Connect to the Weaviate vector database

In [39]:
import weaviate
import json

client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

/opt/conda/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


## Run a similarity search

In [40]:
# Run a similarity search

query_text = "What is an inspector supposed to do?"

response = (
    client.query
    .get(VECTOR_DB_NAME, ["chunk_id", "chunk_text"])
    .with_near_text({"concepts": [query_text]})
    .with_limit(3)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "OHS_ACT_VDB": [
                {
                    "_additional": {
                        "distance": 0.18719542,
                        "id": "8fc6a8fb-5bc6-42a5-9cab-209300822bed"
                    },
                    "chunk_id": 26,
                    "chunk_text": "Functions of inspectors {section title}\u2028\nAn inspector may, for the purposes of this Act-\u2028\nwithout previous notice, at all reasonable times, enter any premises which are occupied or used by an employer or on or in which an employee performs any work or any plant or machinery is used, or which he suspects to be such premises;\u2028\nquestion any person who is or was on or in such premises, either alone or in the presence of any other person, on any matter to which this Act relates;\u2028\nrequire from any person who has control over or custody of a book, record or other document on or in those premises, to produce to him forthwith, or at such time and pl

## Create a natural language answer

In [41]:
# Generative natural language output
# https://weaviate.io/developers/weaviate/quickstart#generative-search-grouped-task
# I don't think ChatGPT is being used for the text generation. It could be some other OpenAi model.

query_text1 = "What is the definition of listed work?"
query_text2 = "What is the definition of listed work?"

response = (
    client.query
    .get(VECTOR_DB_NAME, ["chunk_id", "chunk_text"])
    .with_near_text({"concepts": [query_text1]})
    .with_generate(grouped_task=query_text2)
    .with_limit(3)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "OHS_ACT_VDB": [
                {
                    "_additional": {
                        "distance": 0.17520219,
                        "generate": {
                            "error": null,
                            "groupedResult": "Listed work refers to any work that has been declared as such by the Minister through a notice in the Gazette. The conditions or circumstances under which the work is considered listed work are specified in the notice. Before declaring any work as listed work, the Minister must publish a draft notice in the Gazette and invite interested persons to submit written comments and representations within a specified period. A minimum of three months must elapse between the publication of the draft notice and the final notice.\n\nThere are exceptions to the provisions mentioned above. If the Minister decides to publish the notice in an amended form based on the comments and representations received, subsect

## Use OpenAi to create a natural language response

### Extract the context

In [42]:
# Query the vector database

#query_text = "What are the duties of employees at work?"

query_text = "An accident has happened at my company. What should I do?"

response = (
    client.query
    .get(VECTOR_DB_NAME, ["chunk_id", "chunk_text"])
    .with_near_text({"concepts": [query_text]})
    .with_limit(3)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "OHS_ACT_VDB": [
                {
                    "_additional": {
                        "distance": 0.19975829,
                        "id": "3c6ab759-366a-43eb-9543-6004d9a60272"
                    },
                    "chunk_id": 21,
                    "chunk_text": "Report to inspector regarding certain incidents {section title}\u2028\nEach incident occurring at work or arising out of or in connection with the activities of persons at work, or in connection with the use of plant or machinery, in which, or in consequence of which-\u2028\nany person dies, becomes unconscious, suffers the loss of a limb or part of a limb or is otherwise injured or becomes ill to such a degree that he is likely either to die or to suffer a permanent physical defect or likely to be unable for a period of at least 14 days either to work or to continue with the activity for which he was employed or is usually employed;\u2028\na major incident occurr

In [43]:
# Get the chunks for the context
response_list = response['data']['Get']['OHS_ACT_VDB']

len(response_list)

3

In [44]:
text = response_list[1]['chunk_text']

text

'Report to inspector regarding certain incidents {section title}\u2028\nEach incident occurring at work or arising out of or in connection with the activities of persons at work, or in connection with the use of plant or machinery, in which, or in consequence of which-\u2028\nany person dies, becomes unconscious, suffers the loss of a limb or part of a limb or is otherwise injured or becomes ill to such a degree that he is likely either to die or to suffer a permanent physical defect or likely to be unable for a period of at least 14 days either to work or to continue with the activity for which he was employed or is usually employed;\u2028\na major incident occurred; or\u2028\nthe health or safety of any person was endangered and where-\u2028\na dangerous substance was spilled;\u2028\nthe uncontrolled release of any substance under pressure took place;\u2028\nmachinery or any part thereof fractured or failed resulting in flying, falling or uncontrolled moving objects; or\u2028\nmachin

In [45]:
# Extract just the text

pred_text_list = []

for i in range(0, len(response_list)):
    
    pred_text = response_list[i]['chunk_text']
    
    pred_text_list.append(pred_text)
    
len(pred_text_list)

3

In [46]:
pred_text_list[2]

'General duties of employees at work {section title}\u2028\u2028Every employee shall at work-\u2028\ntake reasonable care for the health and safety of himself and of other persons who may be affected by his acts or omissions;\u2028\nas regards any duty or requirement imposed on his employer or any other person by this Act, co-operate with such employer or person to enable that duty or requirement to be performed or complied with;\u2028\ncarry out any lawful order given to him, and obey the health and safety rules and procedures laid down by his employer or by anyone authorized thereto by his employer, in the interest of health or safety;\u2028\nif any situation which is unsafe or unhealthy comes to his attention, as soon as practicable report such situation to his employer or to the health and safety representative for his workplace or section thereof, as the case may be, who shall report it to the employer; and\u2028\nif he is involved in any incident which may affect his health or wh

### Create the natural language response

In [47]:
context = pred_text_list

# Prepare the prompt
prompt = f"""
Excerpts from the South African Occupational Health and Safety Act (OHS Act): 
{context}
Question: {query_text}

Extract the answer to the question from the text provided. 
If the text doesn't contain the answer, 
reply that the answer is not available."""

query_text

'An accident has happened at my company. What should I do?'

In [48]:
from openai import OpenAI

openai_client = OpenAI(api_key=OPENAI_API_KEY)

completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are a helpful legal assistant who is an expert on the South African OHS Act."},
    {"role": "user", "content": prompt}
  ]
)


print(completion.choices[0].message.content)

Based on the excerpts provided from the South African Occupational Health and Safety Act (OHS Act), in the event of an accident at your company where any person dies, becomes unconscious, suffers a significant injury, or a major incident occurs, the employer or user of the plant or machinery involved must report the incident to an inspector within the prescribed period and manner.

Additionally, if an incident occurs where a person dies, is seriously injured, or suffers the loss of a limb, no one should disturb the site of the incident or remove any article or substance without the consent of an inspector, except to prevent further incidents, rescue individuals, or remove the injured or deceased.

Employees are also required to report any unsafe or unhealthy situation and incidents affecting their health or causing injury to their employer, health and safety representatives, or authorized personnel promptly.

Therefore, in light of an accident at your company, you should ensure that th